# Pull Financial Historicals

## Goal
**Automate pulling financial historicals from North American public companies**

## Setup

First, import the relevant libraries.

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


<div class="alert alert-block alert-info">
<b>Tip:</b> The "headers" below is commonly used in HTTP requests to specify various parameters that control the request behavior and inform the server about the client's environment. 
</div>

**User-Agent:** This header identifies the client software.

>'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
Accept: This header specifies the types of content the client can process.

**'Accept':** This header specifies the types of content the client can process. 

>'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
Accept-Language: This header indicates the preferred languages for the response.

**Accept-Language:** This header indicates the preferred languages for the response.
>'Accept-Language': 'en-CA,en-US;q=0.9,en;q=0.8,zh-CN;q=0.7,zh;q=0.6'
Cache-Control: This header controls caching behavior.

**Cache-Control:** This header controls caching behavior.
>'Cache-Control': 'max-age=0'


**Connection:** This header controls whether the connection stays open after the current transaction.

>'Connection': 'keep-alive'
Upgrade-Insecure-Requests: This header indicates the client's preference for secure connections.

**Upgrade-Insecure-Requests:** This header indicates the client's preference for secure connections.
>'Upgrade-Insecure-Requests': '1'
Accept-Encoding: This header specifies the content-encoding the client can handle.

**Accept-Encoding:** This header specifies the content-encoding the client can handle.
>'Accept-Encoding': 'gzip, deflate, br, zstd'

In [18]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-CA,en-US;q=0.9,en;q=0.8,zh-CN;q=0.7,zh;q=0.6',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Accept-Encoding': 'gzip, deflate, br, zstd'
}


To get your own HTTP headers, particularly the User-Agent, you can inspect the network requests made by your web browser. Here’s how you can do it for different browsers:

**Google Chrome**

- Open the Developer Tools:

    - Right-click on a web page and select "Inspect," or press Ctrl+Shift+I (Windows/Linux) or Cmd+Option+I (Mac).

- Go to the Network Tab:

    - Click on the "Network" tab at the top of the Developer Tools pane.

- Reload the Page:

    - Reload the page (F5 or Ctrl+R) to capture the network requests.

- Select a Request:

    - Click on any request listed in the "Network" tab. The main HTML document request is usually at the top.

- View Headers:

    - In the right-hand pane, click on the "Headers" tab to see the details of the request headers.

<div class="alert alert-block alert-warning">
<b>Important:</b> Note that due to anti-scraping techniques, the website is case sensitive, only lower case ticker can access quarterly.

</div>

**Please type the ticker of the company below in all lower case.**

In [29]:
ticker = 'nvda' 


The different worksheets to scrape are:

In [33]:

urls = {}
urls['income annually'] = f"https://stockanalysis.com/stocks/{ticker}/financials/"
urls['income quarterly'] = f"https://stockanalysis.com/stocks/{ticker}/financials/?p=quarterly"
urls['balance sheet annually'] = f"https://stockanalysis.com/stocks/{ticker}/financials/balance-sheet/"
urls['balance sheet quarterly'] = f"https://stockanalysis.com/stocks/{ticker}/financials/balance-sheet/?p=quarterly"
urls['cash flow annually'] = f"https://stockanalysis.com/stocks/{ticker}/financials/cash-flow-statement/"
urls['cash flow quarterly'] = f"https://stockanalysis.com/stocks/{ticker}/financials/cash-flow-statement/?p=quarterly"
urls['ratio annually'] = f"https://stockanalysis.com/stocks/{ticker}/financials/ratios/"
urls['ratio quarterly'] = f"https://stockanalysis.com/stocks/{ticker}/financials/ratios/?p=quarterly"

# feel free to comment out ones that you do not want or reorder them

In [34]:
filename = f'financial statements ({ticker}).xlsx'


In [35]:
with pd.ExcelWriter(filename, engine='xlsxwriter') as xlwriter:

    for key in urls.keys():
        response = requests.get(urls[key], headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        df = pd.read_html(str(soup), attrs={'data-test': 'financials'})[0]

        # drop the last column
        df = df.iloc[:, :-1]

        # Split the DataFrame into the first column and the remaining columns
        first_column = df.iloc[:, 0]
        remaining_columns = df.iloc[:, 1:]
            
        # Reverse the order of the remaining columns
        remaining_columns = remaining_columns.iloc[:, ::-1]
            
        # Concatenate the first column with the reversed remaining columns
        df = pd.concat([first_column, remaining_columns], axis=1)

        df.to_excel(xlwriter, sheet_name=key, index=False)

        worksheet = xlwriter.sheets[key]
        for i, col in enumerate(df.columns):
            max_length = max(df[col].astype(str).map(len).max(), len(col)) + 2  # Adding a little extra space
            worksheet.set_column(i, i, max_length)
 



C:\Users\USER\AppData\Local\Temp\ipykernel_15076\3807159506.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs={'data-test': 'financials'})[0]
C:\Users\USER\AppData\Local\Temp\ipykernel_15076\3807159506.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs={'data-test': 'financials'})[0]
C:\Users\USER\AppData\Local\Temp\ipykernel_15076\3807159506.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs={'data-test': 'financials'})[0]
C:\Users\USER\AppData\Local\Temp\ipykernel_15076\3807159506.py:6: FutureWarning: Passing

<div class="alert alert-block alert-success">
<b>Up to you:</b> Congratulations! You will now see the extracted file saved in the same directory this program is downlowded at. 
</div>

The methodology and implementation due credit to Jie Jenn. 
https://youtu.be/tr_NQVUdmvY?si=va25lutZmMWGZC_C
